# ロジスティック回帰
- 「回帰」という名前ではあるが分類問題を解くものである
- このアルゴリズムを実装していく

# 2値分類

ここからは出力ベクトル$y$は連続値から0もしくは1の値となる。
$$ y \in \{0, 1\}$$
このとき0は陰性、1は陽性などとして表現されるがいずれも定まった表現は無いとのこと。

このように二つの分類を考えることを「二値分類問題」と呼ぶ。

今までのように線形回帰を使ってこの問題を解く場合にはあらゆる予測結果をマッピングしておいて、それが0.5を越えれば1、越えなければ0と分類するやり方が一つ考えられるだろう。

しかしながらこの方法ではこの問題を上手く解くことができない。その原因は多くの問題が非線形であることに起因する。

## 仮説表現

我々の仮説は以下を満たすものとする。要はこの仮説が出力するのは確率であると言える。

$$0 \leq h_\theta(x) \leq 1$$

このとき「シグモイド関数」という新たな形式を取り入れると、仮説表現は以下のようになる。

- $ h_\theta(x) = g(\theta^Tx)$ : 任意の実数$\theta^Tx$の$g$という関数による写像が$h_\theta(x)$   
- $ z = \theta^Tx $ : 特徴量$\theta$とデータ$x$の積を$z$により抽象化  
- $ g(z) = \frac{1}{1 + e^{-z}} $ : $g$による$z$の写像がシグモイド関数またはロジスティック関数  

![https://en.wikipedia.org/wiki/Logistic_function](logistic-function.png)  

この$g(z)$はどんな実数も$\{0, 1\}$区間にマップするため任意の値を持つ関数を分類に適したより良い関数に変換することができる。1に近ければ1だし、0に近ければ0という容易な判断ができるようになった。

たとえば$g(z)$は結果が1と判断できるように確率$h_\theta(x) = 0.7 $を我々に与えてくれる。

結果が1もしくは0と判断できる確率は以下のように表現される。

$$ h_\theta(x) = P(y = 1 | x; \theta) = 1 - P(y = 0 | x; \theta) $$
$$ P(y = 0 | x; \theta) + P(y = 1 | x; \theta) = 1 $$

これはつまり予測結果が1の確率が70%ならば0である確率は30%であるということだ。



### ※YOUR CODE IS HERE

In [1]:
import numpy as np

def sigmoid(X):
    return

### test cases

In [14]:
np.testing.assert_allclose(sigmoid(-5), 0.0066929, rtol=1e-05)
np.testing.assert_allclose(sigmoid(0), 0.50000, rtol=1e-05)
np.testing.assert_allclose(sigmoid(5), 0.99331, rtol=1e-05)

np.testing.assert_allclose(
    sigmoid(np.array([4, 5, 6])), np.array([0.98201, 0.99331, 0.99753]), rtol=1e-05)

np.testing.assert_allclose(
    sigmoid(np.array([[-1], [0], [1]])), np.array([[0.26894], [0.50000], [0.73106]]), rtol=1e-05)

## 決定境界


0か1かの分類結果を得るために我々は以下の仮説表現に従って得られた確率から0/1を判断することができる。

$$ h_\theta(x) \geq 0.5 \rightarrow y = 1 $$

$$ h_\theta(x) < 0.5 \rightarrow y = 0 $$

$g$はその入力が0以上であるときその出力が0.5以上となるなど、その振る舞いは以下のように定義される。

- $ z = 0, e^0 = 1 \Rightarrow g(z) = 1/2 $
- $ z \rightarrow \infty, e^{-\infty} \rightarrow 0 \Rightarrow g(z) = 1 $
- $ z \rightarrow -\infty, e^{\infty} \rightarrow \infty \Rightarrow g(z) = 0 $

つまりgに対する入力が$\theta^TX$ならば

$$h_\theta(x) = g(\theta^Tx) \geq 0.5 $$
$$ when \ \ \theta^Tx \geq 0 $$

よって以下が言える

$$\theta^Tx \geq 0 \Rightarrow y = 1 $$
$$\theta^Tx \leq 0 \Rightarrow y = 0 $$

そして決定境界は$y=0$と$y=1$となるエリアを分割する線のことであり、以下のような仮説関数によって定義される。

$$ \theta = \begin{bmatrix} 5 \\ -1 \\ 0 \end{bmatrix}$$
$$ y = 1 \ \ if \ \  5 + (-1)x_1 + 0x_2 \geq 0 $$
$$ 5 - x_1 \geq 0 $$
$$ -x_1 \geq -5 $$
$$ x_1 \leq 5 $$

この場合、決定境界は垂直な直線で$x_1 = 5$で表現され、その左は全て$y = 1$右は$y=0$となる。

そしてシグモイド関数$g(z)$への入力は線形である必要がなく円形（$e.g. z = \theta_0 + \theta_1x_1^2+\theta_2x_2^2$）のような形でも構わない。

## コスト関数

ここでも線形回帰とは異なる関数を使う必要がある。何故ならばロジスティック関数の出力は波状であり多くの局所最適解が存在するためである。つまり凸関数では無いということだ。代わりに以下のコスト関数を使う。

$$J(\theta) = \frac{1}{m} \sum_{i=1}^{m}Cost(h_\theta(x^{(i)}), y^{(i)})$$
$$ Cost(h_\theta(x), y) = -log(h_\theta(x)) \ \ if \ \ y=1 $$
$$ Cost(h_\theta(x), y) = -log((1 - h_\theta(x)) \ \ if \ \ y=0 $$

![y=1](lr-cost-function-y1.png)
![y=0](lr-cost-function-y0.png)

このコスト関数の見方は仮説の予測結果がyから離れれば離れるほどコスト関数の出力は大きくなる。逆に予測結果がyに等しいとコスト関数の出力は0に近づく。

$$ Cost(h_\theta(x), y) = 0 \ \ if \ \ h_\theta(x) = y $$

また$y$が0の場合、仮説関数も0を出力すればコスト関数は0になります。仮説が1に近づけば、コスト関数は無限大に近づくことになります。

$$ Cost(h_\theta(x), y) \rightarrow \infty \ \ if \ \ y=0 \ and \ h_\theta(x) \rightarrow 1 $$

$y$ が 1の場合、仮説関数が1を出力すればコスト関数は 0 になります。我々の仮説が 0 に近づけば、コスト関数は無限大に近づきます。

$$ Cost(h_\theta(x), y) \rightarrow \infty \ \ if \ \ y=1 \ and \ h_\theta(x) \rightarrow 0 $$

## 単純化したコスト関数と勾配効果法

これまでコスト関数を二つに場合分けしていたが、以下のように一つにまとめることができる。

$$ Cost(h_\theta(x), y) = -ylog(h_\theta(x)) - (1 - y)log(1 - h_\theta(x)) $$

上式から$y$が1の時は第二項$(1 - y)log(1 - h_\theta(x))$は0となり、もし$y$が0の時は第一項$-ylog(h_\theta(x))$が0になることに注目されたい。

以上よりコスト関数全体を以下のように表現することができる。

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^{m}[y^{(i)}log(h_\theta(x^{(i)})) + (1-y^{(i)})log(1-h_\theta(x^{(i)}))]$$

ベクトル化による実装は以下のようになる。

$$ h = g(X\theta) $$
$$ J(\theta) = \frac{1}{m} \cdot (-y^T log(h) - (1 - y)^T log(1 - h)) $$

### 勾配効果法

勾配効果法の一般的な式は以下のようなものであったことを思い出してほしい。

$$ Repeat\ \  {\theta_j := \theta_j - \alpha \frac{\partial}{\partial \theta_j} J(\theta)} $$

微分の部分を計算すると、次のようになる。

$$\begin{align*}
& Repeat \; \lbrace \newline
& \; \theta_j := \theta_j - \frac{\alpha}{m} \sum_{i=1}^m (h_\theta(x^{(i)}) - y^{(i)}) x_j^{(i)} \newline & \rbrace
\end{align*}$$

これは線形回帰で使用していたものと同じであり、$\theta$の全ての値を同時に更新する必要がある。

ベクトル化による実装は以下のようになる。

$$ \theta := \theta - \frac{\alpha}{m}X^T(g(X\theta) - \overset{\rightarrow}{y}) $$

まずシグモイド関数の微分を計算します（これは$J(\theta)$の偏微分を求めるときに役立ちます）。

$\begin{align*}\sigma(x)'&=\left(\frac{1}{1+e^{-x}}\right)'=\frac{-(1+e^{-x})'}{(1+e^{-x})^2}=\frac{-1'-(e^{-x})'}{(1+e^{-x})^2}=\frac{0-(-x)'(e^{-x})}{(1+e^{-x})^2}=\frac{-(-1)(e^{-x})}{(1+e^{-x})^2}=\frac{e^{-x}}{(1+e^{-x})^2} \newline &=\left(\frac{1}{1+e^{-x}}\right)\left(\frac{e^{-x}}{1+e^{-x}}\right)=\sigma(x)\left(\frac{+1-1 + e^{-x}}{1+e^{-x}}\right)=\sigma(x)\left(\frac{1 + e^{-x}}{1+e^{-x}} - \frac{1}{1+e^{-x}}\right)=\sigma(x)(1 - \sigma(x))\end{align*}$

これで、結果として得られる偏微分を求める準備ができました。

$\begin{align*}\frac{\partial}{\partial \theta_j} J(\theta) &= \frac{\partial}{\partial \theta_j} \frac{-1}{m}\sum_{i=1}^m \left [ y^{(i)} log (h_\theta(x^{(i)})) + (1-y^{(i)}) log (1 - h_\theta(x^{(i)})) \right ] \newline&= - \frac{1}{m}\sum_{i=1}^m \left [     y^{(i)} \frac{\partial}{\partial \theta_j} log (h_\theta(x^{(i)}))   + (1-y^{(i)}) \frac{\partial}{\partial \theta_j} log (1 - h_\theta(x^{(i)}))\right ] \newline&= - \frac{1}{m}\sum_{i=1}^m \left [     \frac{y^{(i)} \frac{\partial}{\partial \theta_j} h_\theta(x^{(i)})}{h_\theta(x^{(i)})}   + \frac{(1-y^{(i)})\frac{\partial}{\partial \theta_j} (1 - h_\theta(x^{(i)}))}{1 - h_\theta(x^{(i)})}\right ] \newline&= - \frac{1}{m}\sum_{i=1}^m \left [     \frac{y^{(i)} \frac{\partial}{\partial \theta_j} \sigma(\theta^T x^{(i)})}{h_\theta(x^{(i)})}   + \frac{(1-y^{(i)})\frac{\partial}{\partial \theta_j} (1 - \sigma(\theta^T x^{(i)}))}{1 - h_\theta(x^{(i)})}\right ] \newline&= - \frac{1}{m}\sum_{i=1}^m \left [     \frac{y^{(i)} \sigma(\theta^T x^{(i)}) (1 - \sigma(\theta^T x^{(i)})) \frac{\partial}{\partial \theta_j} \theta^T x^{(i)}}{h_\theta(x^{(i)})}   + \frac{- (1-y^{(i)}) \sigma(\theta^T x^{(i)}) (1 - \sigma(\theta^T x^{(i)})) \frac{\partial}{\partial \theta_j} \theta^T x^{(i)}}{1 - h_\theta(x^{(i)})}\right ] \newline&= - \frac{1}{m}\sum_{i=1}^m \left [     \frac{y^{(i)} h_\theta(x^{(i)}) (1 - h_\theta(x^{(i)})) \frac{\partial}{\partial \theta_j} \theta^T x^{(i)}}{h_\theta(x^{(i)})}   - \frac{(1-y^{(i)}) h_\theta(x^{(i)}) (1 - h_\theta(x^{(i)})) \frac{\partial}{\partial \theta_j} \theta^T x^{(i)}}{1 - h_\theta(x^{(i)})}\right ] \newline&= - \frac{1}{m}\sum_{i=1}^m \left [     y^{(i)} (1 - h_\theta(x^{(i)})) x^{(i)}_j - (1-y^{(i)}) h_\theta(x^{(i)}) x^{(i)}_j\right ] \newline&= - \frac{1}{m}\sum_{i=1}^m \left [     y^{(i)} (1 - h_\theta(x^{(i)})) - (1-y^{(i)}) h_\theta(x^{(i)}) \right ] x^{(i)}_j \newline&= - \frac{1}{m}\sum_{i=1}^m \left [     y^{(i)} - y^{(i)} h_\theta(x^{(i)}) - h_\theta(x^{(i)}) + y^{(i)} h_\theta(x^{(i)}) \right ] x^{(i)}_j \newline&= - \frac{1}{m}\sum_{i=1}^m \left [ y^{(i)} - h_\theta(x^{(i)}) \right ] x^{(i)}_j  \newline&= \frac{1}{m}\sum_{i=1}^m \left [ h_\theta(x^{(i)}) - y^{(i)} \right ] x^{(i)}_j\end{align*}$

このベクトル化実装は以下のようになる。

$$  \nabla J(\theta) = \frac{1}{m} \cdot X^T \cdot (g(X \cdot \theta) - \overset{\rightarrow}{y})  $$

### ※YOUR CODE IS HERE

In [ ]:
def lrCostFunction(X, y, theta):
    return J, grad

### test cases

In [28]:
theta = np.array([[-2],[-1], [1], [2]])
X = np.array([
    [0.1, 0.6, 1.1],
    [0.2, 0.7, 1.2],
    [0.3, 0.8, 1.3],
    [0.4, 0.9, 1.4],
    [0.5, 1.0, 1.5],
])
X = np.hstack((np.ones((5, 1)), X))
y = np.array([[1], [0], [1], [0], [1]])

J, grad = lrCostFunction(X, y, theta)

np.testing.assert_allclose(J, np.array([0.73482]), rtol=1e-05)
np.testing.assert_allclose(
    grad, np.array([[0.14656137],[0.05144159],[0.12472227],[0.19800296]]), rtol=1e-05)

## ここまででex2の演習に一旦移動

## 特徴量正規化

それぞれの入力値をほぼ同じ範囲にすることで、勾配降下法を高速化することができます。これは、θが小さい範囲では素早く、大きい範囲ではゆっくりと下降するため、変数が非常に不均一な場合、最適な値まで非効率的に振動してしまうためです。

これを防ぐためには、入力変数の範囲を変更して、すべての変数がほぼ同じになるようにすることです。理想的には以下となる

$-1 \leq x_{(i)} \leq 1 \ \ or \ \ -0.5 \leq x_{(i)} \leq 0.5 $

これらは厳密な要件ではなく、あくまでもスピードアップを図るためのものです。目標は、すべての入力変数を、多少の差はあれど、大体これらの範囲に収めることです。

そのための手法として、特徴量のスケーリングと平均値の正規化があります。特徴量スケーリングでは、入力値を入力変数の範囲（すなわち、最大値から最小値を引いたもの）で割ることで、新しい範囲が1になります。平均正規化では、入力変数の値からその入力変数の平均値を引くことで、入力変数の新しい平均値が0になります。この2つの手法を実行するには、次の式のように入力値を調整します。

$ x_i := \frac{x_i - \mu_i}{s_i}$

ここで、$\mu_i$ は特徴量（i）のすべての値の平均値、$s_i$ は値の範囲（最大-最小）、または $s_i$は標準偏差です。

範囲で割るのと、標準偏差で割るのとでは、結果が異なることに注意してください。以下は範囲を使った例ですが演習では標準偏差を使用しています。

例：$x_i$は住宅価格で、範囲は100から2000、平均値は1000です。そして，$x_i := \frac{price-1000}{1900}$



#### ※ YOUR CODE IS HERE

In [ ]:
def featureNormalize(X):
    return 

np.testing.assert_equal(featureNormalize(np.array([[1], [2], [3]])), np.array([[-1.], [0.], [1.]]))

#### ※ END

## 更なる最適化

「共役勾配」、「BFGS」、「L-BFGS」は、勾配降下法の代わりに使用できる、より洗練された高速なθの最適化方法です。A.Ngは，これらのより洗練されたアルゴリズムを自分で書かずに（数値計算の専門家でない限り），代わりにライブラリを使うことを提案しています。なぜなら，それらはすでにテストされ，高度に最適化されているからです。Octaveはそれらを提供します。

まず，与えられた入力値θに対して，以下の2つの関数を評価する関数を用意する必要があります。

## 多クラス分類

## 正則化、コスト関数

## 正則化線形回帰

## 正則化ロジスティック回帰

## 初期の単位特徴ベクトル